Инициализация.
---

In [1]:
import sys
cadEditorDir = "d:/DEV/MYGIT/CadEditor/" #test at CadEditor v3.5
sys.path.append(cadEditorDir)
import clr
clr.AddReference("CadEditor")
from CadEditor import Globals, ConfigScript, Utils, OffsetRec
from array import array

romName    = cadEditorDir + "Flintstones, The - The Rescue of Dino & Hoppy (U) [!].nes"

Подсчёт количества макроблоков на уровне
---
* Макроблоки в игре нумеруются нестандартно - сначала увеличиваются первые 4 бита номера, потом разряд перебрасывается в младшие биты. Т.е. порядок выглядит так: 0x00, 0x10, 0x20, 0x30, ... , 0xD0, 0xE0, 0xF0, 0x01, 0x11, 0x21, 0x31, ...,0x,D1, 0xE1, 0xF1, 0x02, 0x12, 0x22, ..., 0xDF, 0xDE, 0xDF
Соответсвенно, количество макроблоков на уровне можно вычислить как разницу между максимально возможным числом (256) и количеством невстречаемых элементов.

In [2]:
def invertNibbles(v):
    return v >> 4 | (v & 0x0F) << 4

def getMaxMacroBlockIndex(indexList):
    maxIndexInv = 0x00
    for index in indexList:
        indexInv = invertNibbles(index)
        if indexInv > maxIndexInv:
            maxIndexInv = indexInv
    return invertNibbles(maxIndexInv)

def calcMacroBlocksCount(indexList) :
    return invertNibbles(getMaxMacroBlockIndex(indexList))+1

Рассчёт количества блоков (обычная арифметика)

In [3]:
def calcMacroBlockAddr():
    offRec = ConfigScript.screensOffset[0]
    return  offRec.beginAddr +  offRec.width * offRec.height

def readMacroBlocksFromFromRom(mbCount):
    rom = array("i", Globals.romdata)
    mbAddr = calcMacroBlockAddr()
    mbData = rom[mbAddr:mbAddr+mbCount*16]
    return mbData

def calcBlocksCount(mbData):
    return max(mbData)+1

def calcBlockAddr(mbCount):
    return calcMacroBlockAddr() + 16*mbCount

def calcBlockSubpalAddr(mbCount, bCount):
    return calcBlockAddr(mbCount) + bCount * 4

Подсчёт для всех конфигов игры

In [4]:
from IPython.display import display
from ipy_table import *

for i in xrange(7):
    levelNo = i+1
    configName = cadEditorDir + "settings_flintstones_rescue_of_dino_and_hoppy/Settings_Flintstones-%d.cs"%levelNo
    Globals.loadData(romName, "", configName) #reload Globals & ConfigScript for current level
    screenData = Utils.setScreens(0)[0]       #read screen 0 for level 0 in config
    a = array("i", screenData)                #load data from .NET array to python array
    toTable = []
    mbCount = calcMacroBlocksCount(a)
    print "Количество макроблоков для уровня %d :"%levelNo
    print "  Макс. индекс:%s"%hex(invertNibbles(mbCount-1))
    #закомментировать для блог записи, вывод информации для базы IDA
    mbAddr = calcMacroBlockAddr()
    mbData = readMacroBlocksFromFromRom(mbCount)
    toTable.append([u"Уровень %d"%levelNo, u"Адрес", u"Кол-во", u"Размер"])
    toTable.append([u"Макроблоки", u"%8s"%hex(mbAddr), u"%4d"%mbCount, u"%4d"%len(mbData)])
    bCount = calcBlocksCount(mbData)
    toTable.append([u"Блоки", u"%8s"%hex(calcBlockAddr(mbCount)),u"%4d"%bCount, u"%4d"%(bCount*4)])
    toTable.append([u"Аттр.блоков",u"%8s"%hex(calcBlockSubpalAddr(mbCount, bCount)), u"%4d"%bCount, u"%4d"%bCount])
    t = make_table(toTable)
    t.apply_theme("basic")
    display(t)
    

Количество макроблоков для уровня 1 :
  Макс. индекс:0xdf


Уровень 1,Адрес,Кол-во,Размер
Макроблоки,0x1f0,254,4064
Блоки,0x11d0,238,952
Аттр.блоков,0x1588,238,238


Количество макроблоков для уровня 2 :
  Макс. индекс:0xcf


Уровень 2,Адрес,Кол-во,Размер
Макроблоки,0x1719,253,4048
Блоки,0x26e9,256,1024
Аттр.блоков,0x2ae9,256,256


Количество макроблоков для уровня 3 :
  Макс. индекс:0x4f


...

Уровень 5,Адрес,Кол-во,Размер
Макроблоки,0x31f4,243,3888
Блоки,0x4124,256,1024
Аттр.блоков,0x4524,256,256


Количество макроблоков для уровня 6 :
  Макс. индекс:0xcf


Уровень 6,Адрес,Кол-во,Размер
Макроблоки,0x4190,253,4048
Блоки,0x5160,256,1024
Аттр.блоков,0x5560,256,256


Количество макроблоков для уровня 7 :
  Макс. индекс:0xf


Уровень 7,Адрес,Кол-во,Размер
Макроблоки,0x3b34,241,3856
Блоки,0x4a44,256,1024
Аттр.блоков,0x4e44,256,256


То же самое для уровня, для которого нет конфига, а известен только стартовый адрес и размер экрана

In [5]:
addr = 0x52A9
size = 384
#addr = 0x604c
#size = 8*32
rom = array("i", Globals.romdata)

a = rom[addr:addr+size]

toTable = []
mbCount = calcMacroBlocksCount(a)
print "Количество макроблоков для уровня X"
print "  Макс. индекс:%s"%hex(invertNibbles(mbCount-1))
#закомментировать для блог записи, вывод информации для базы IDA
#---
mbAddr = addr + size
mbData = rom[mbAddr:mbAddr+mbCount*16]
toTable.append([u"Уровень X", u"Адрес", u"Кол-во", u"Размер"])
toTable.append([u"Макроблоки", u"%8s"%hex(mbAddr), u"%4d"%mbCount, u"%4d"%len(mbData)])
#---
bCount = calcBlocksCount(mbData)
bAddr = mbAddr + mbCount*16
toTable.append([u"Блоки", u"%8s"%hex(bAddr),u"%4d"%bCount, u"%4d"%(bCount*4)])
toTable.append([u"Аттр.блоков",u"%8s"%hex(bAddr+4*bCount), u"%4d"%bCount, u"%4d"%bCount])
t = make_table(toTable)
t.apply_theme("basic")
display(t)

Количество макроблоков для уровня X
  Макс. индекс:0x78


Уровень X,Адрес,Кол-во,Размер
Макроблоки,0x5429,136,2176
Блоки,0x5ca9,179,716
Аттр.блоков,0x5f75,179,179


Flinstones: The Surprise at Dinosaur Peak!
---
А теперь для Flinstones: The Surprise at Dinosaur Peak!<br>
Отличаются только размер макроблока 4x2, и следовательно порядок макроблоков (первыми меняются старшие 3 бита, далее инкрементируются в лексикографическом порядке младшие 5 бит.

In [6]:
def invertNibbles3(v):
    return v >> 5 | (v & 0x1F) << 3

def invertNibbles5(v):
    return v >> 3 | (v & 0x07) << 5

def getMaxMacroBlockIndex3(indexList):
    maxIndexInv = 0x00
    for index in indexList:
        indexInv = invertNibbles5(index)
        if indexInv > maxIndexInv:
            maxIndexInv = indexInv
    return invertNibbles3(maxIndexInv)

def calcMacroBlocksCount3(indexList) :
    return invertNibbles5(getMaxMacroBlockIndex3(indexList))+1

print calcMacroBlocksCount3([0x43])

105


In [7]:
addr = 0xD833
size = 8*96

romName    = cadEditorDir + "Flintstones, The - The Surprise at Dinosaur Peak! (U) [!p].nes"
configName = cadEditorDir + "settings_flintstones_surprise_at_dinosaur_peak/Settings_Flintstones2-%d.cs"%1
Globals.loadData(romName, "", configName) #reload Globals & ConfigScript for current level

rom = array("i", Globals.romdata)

a = rom[addr:addr+size]

toTable = []
mbCount = calcMacroBlocksCount3(a)
print "Количество макроблоков для уровня X"
print "  Макс. индекс:%s"%hex(invertNibbles3(mbCount-1))
#закомментировать для блог записи, вывод информации для базы IDA
#---
mbAddr = addr + size
mbData = rom[mbAddr:mbAddr+mbCount*8]
toTable.append([u"Уровень X", u"Адрес", u"Кол-во", u"Размер"])
toTable.append([u"Макроблоки", u"%8s"%hex(mbAddr), u"%4d"%mbCount, u"%4d"%len(mbData)])
#---
bCount = calcBlocksCount(mbData)
bAddr = mbAddr + mbCount*8
toTable.append([u"Блоки", u"%8s"%hex(bAddr),u"%4d"%bCount, u"%4d"%(bCount*4)])
toTable.append([u"Аттр.блоков",u"%8s"%hex(bAddr+4*bCount), u"%4d"%bCount, u"%4d"%bCount])
t = make_table(toTable)
t.apply_theme("basic")
display(t)

Количество макроблоков для уровня X
  Макс. индекс:0x43


Уровень X,Адрес,Кол-во,Размер
Макроблоки,0xdb33,105,840
Блоки,0xde7b,162,648
Аттр.блоков,0xe103,162,162
